In [ ]:
# Try to install the packages within the current kernel
%pip install pytube openai streamlit moviepy yt-dlp langchain_Groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install pytube3 --upgrade
from pytube import YouTube # Change to YouTube instead of Youtube
import yt_dlp
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-0ou_o3zd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-0ou_o3zd
  Resolved https://github.com/openai/whisper.git to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import yt_dlp
import whisper
import subprocess
import os

def download_audio(video_url):
    """Download YouTube audio and save as an MP3 file"""
    output_file = "video_Audio.mp3"  # Correct output filename

    # 🔹 Ensure no old files are present
    if os.path.exists(output_file):
        os.remove(output_file)  # Delete old MP3 file

    ydl_opts = {
        'format': 'bestaudio',
        'outtmpl': 'video_Audio.%(ext)s',  # Dynamic extension
        'noplaylist': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',  # Convert to MP3 to avoid Whisper issues with MP4
            'preferredquality': '192',
        }],
        'overwrites': True,  # Force overwrite to prevent old files from being reused
        # 🔹 Increase timeout settings
        'socket_timeout': 60,  # Increase socket timeout to 60 seconds
        'retries': 10,       # Increase the number of retries to 10
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

    return output_file  # Return the correct MP3 file path

def trim_audio(input_file, output_file="trimmed_audio.mp3", start_time=0, duration=600):
    """
    Trim audio to reduce duration.

    Params:
        input_file: Path to original audio file (MP3)
        output_file: Path to save trimmed audio
        start_time: Start time in seconds (default: 0)
        duration: Clip duration in seconds (default: 300s / 5 minutes)

    Returns:
        Path to trimmed audio file
    """
    command = f"ffmpeg -i {input_file} -ss {start_time} -t {duration} -acodec copy {output_file} -y"
    subprocess.run(command, shell=True, check=True)
    return output_file  # Return new file path

In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import torch
def transcribe_audio(audio_path):
    """Convert speech to text using Whisper AI with GPU acceleration."""
    model = whisper.load_model("small").to("cuda" if torch.cuda.is_available() else "cpu")

    # 🔹 Clear Whisper cache to ensure fresh processing
    os.system("rm -rf ~/.cache/whisper")

    transcription = model.transcribe(audio_path, fp16=True, beam_size=2)
    return transcription["text"]

# 🔹 Step 1: Download Audio from YouTube
video_url = "https://www.youtube.com/watch?v=rz40ukZ3krQ"  # Replace with actual video link
audio_path_mp3 = download_audio(video_url)

# 🔹 Step 2: Trim the audio to 5 minutes
trimmed_audio_path = trim_audio(audio_path_mp3, duration=300)  # Trim to 5 mins

# 🔹 Step 3: Transcribe MP3 using Whisper AI
transcribed_text = transcribe_audio(trimmed_audio_path)  # Use trimmed audio

# 🔹 Step 4: Print and Save the Transcription
print("🔹 Transcription:")
print(transcribed_text)

with open("transcription.txt", "w") as file:
    file.write(transcribed_text)

print("✅ Transcription saved as transcription.txt")

[youtube] Extracting URL: https://www.youtube.com/watch?v=rz40ukZ3krQ
[youtube] rz40ukZ3krQ: Downloading webpage
[youtube] rz40ukZ3krQ: Downloading tv client config
[youtube] rz40ukZ3krQ: Downloading player 20830619
[youtube] rz40ukZ3krQ: Downloading tv player API JSON
[youtube] rz40ukZ3krQ: Downloading ios player API JSON
[youtube] rz40ukZ3krQ: Downloading m3u8 information
[info] rz40ukZ3krQ: Downloading 1 format(s): 251
[download] Destination: video_Audio.webm
[download] 100% of   21.37MiB in 00:00:00 at 40.11MiB/s  
[ExtractAudio] Destination: video_Audio.mp3
Deleting original file video_Audio.webm (pass -k to keep)
🔹 Transcription:
 In this video, I'll show you how to build your own artificial intelligence application that utilizes RAG, Retrieval Augmented Generation, without writing a single line of code, and in just a few minutes. Now the way we'll do that is using something called Langflow. Now I have it open on my screen, you can see what the finished application will look like

In [ ]:
from langchain_groq import ChatGroq

# Replace with your actual Groq API key this is openai so change with the groq api
groq_api_key = "sk-proj-FGmuHz9d--TGrwpdc9Wm5bFIkSyOphOWUS2yGiW5ze3-uXBPiAZP6xFjHQyH8YWQXdsRR2Th8KT3BlbkFJVoT8qWghp_lQtLozwXAWv1fZ2lc5y"

# Initialize the ChatGroq client with a valid model
groq = ChatGroq(api_key=groq_api_key, model="llama-3.3-70b-versatile")

def summarize_text(text):
    """Summarize text using the specified Groq model"""
    response = groq.invoke(
        input=f"Summarize this transcript:\n{text}",
    )
    return response.content

# Example usage
transcribed_text = "This is an example transcript that needs to be summarized."

summary = summarize_text(transcribed_text)
print("🔹 Summary:")
print(summary)


🔹 Summary:
The provided transcript is a single sentence stating it's an example transcript that needs to be summarized. There is no actual content to summarize.


In [ ]:
#make gradio interface for this video summarizer

In [ ]:
# """from google.colab import auth
# auth.authenticate_user()

# !git config --global user.email "harshitsb2615@gmail.com"
# !git config --global user.name "Harshit6057"  """


ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download